In [1]:
%%capture
%load_ext autoreload
%autoreload 2
from setup_nb_env import *

from epsampling.utils import load_csv
# pd.set_option('display.float_format', lambda x: '%.3f' % x)
from epsampling.utils import drop_sers_with_nans
from epsampling.utils import date_str_to_int

DATA_DIR = '/work/users/k/4/k4thryn/Repos/EpSampling/data/'
DT = datetime.today().strftime('%Y%m%d-%H%M%S')

### <font color=blue> Read in deaths df and acs df.

In [10]:
from epsampling.utils import drop_duplicate_cols

timestamp = '20241003-131323'

fpath = os.path.join(DATA_DIR,'processed','for_modeling',f'df_deaths_{timestamp}.csv')
df_deaths = pd.read_csv(fpath)
fpath = os.path.join(DATA_DIR,'processed','for_modeling',f'df_acs_{timestamp}.csv')
df_acs = pd.read_csv(fpath ,index_col='Fips')

## join on fips.
# df = df_deaths.join(df_acs,on='Fips')
# df

### <font color=blue> Remove highly correlated features from acs df.

In [11]:
# # # # # # # # # # # # # # 
corr_thresh = 0.95
# # # # # # # # # # # # # # 

corr_matrix = df_acs.corr().abs()
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
to_drop = [column for column in upper.columns if any(upper[column] > corr_thresh)]
df_filt = df_acs.drop(to_drop, axis=1, inplace=False)
df_filt
# df_acs = df_filt.copy()
# df_acs

,OCC_MBSA_ELCAM_L,IND_AFFHM_MQE,IND_PSMAW_M,HU_OCC_OPRgt200,HU_UIS50P,IND_WHT_x7,HU_UIS2049,HH_7pP,OCC_MBSA_CES_CM,OCC_MBSA_ELCAM_ADESM,...,HU_UISOTHER,HU_UIS0304,OCC_NRCM_FFF_x11,HU_VAC,IND_AFFHM,POP_OTH2PLNH,IND_PUBA_x7,POP_ASIANNH,POP_AIANNH,POP_AIAN
Fips,,,,,,,,,,,,,,,,,,,,,
1001,70,50,61,56,291,783,287,187,460,246,...,4200,400,145,2314,200,1684,2363,647,98,98
1003,1185,226,24,129,14074,2255,2511,659,1675,1959,...,12598,2383,305,34573,1827,5940,5358,2175,1076,1259
1005,43,0,0,16,56,105,42,84,100,13,...,3512,493,333,2579,469,703,611,106,71,77
1007,14,184,0,18,86,254,19,64,25,80,...,2967,27,37,1930,299,210,433,46,21,21
1009,163,90,0,9,1,893,132,233,345,160,...,6109,378,183,3227,443,1571,819,216,70,189
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56037,165,3135,0,30,73,561,404,164,302,110,...,3830,1023,25,3645,3231,988,1295,448,454,506
56039,209,67,0,74,97,175,404,0,179,601,...,438,1098,92,3724,385,617,401,369,30,30
56041,0,748,0,8,62,116,125,98,30,67,...,1806,237,106,1144,884,571,472,37,20,52


### <font color=blue> Join naive deaths df with filtered acs df.

In [13]:
df = df_deaths.merge(df_filt, on='Fips')
display(df)

,Fips,State_fips,Pop,State_pop,Ratio,Date,Proj_inc_deaths,True_cum_deaths,Cum_deaths_tm1,True_inc_deaths,...,HU_UISOTHER,HU_UIS0304,OCC_NRCM_FFF_x11,HU_VAC,IND_AFFHM,POP_OTH2PLNH,IND_PUBA_x7,POP_ASIANNH,POP_AIANNH,POP_AIAN
0,1001,1,58239,4997675,0.011653,20200620,83.302156,9.0,6.0,3.0,...,4200,400,145,2314,200,1684,2363,647,98,98
1,1001,1,58239,4997675,0.011653,20200627,68.179357,12.0,9.0,3.0,...,4200,400,145,2314,200,1684,2363,647,98,98
2,1001,1,58239,4997675,0.011653,20200704,76.238975,13.0,12.0,1.0,...,4200,400,145,2314,200,1684,2363,647,98,98
3,1001,1,58239,4997675,0.011653,20200711,88.362530,15.0,13.0,2.0,...,4200,400,145,2314,200,1684,2363,647,98,98
4,1001,1,58239,4997675,0.011653,20200718,114.279381,21.0,15.0,6.0,...,4200,400,145,2314,200,1684,2363,647,98,98
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
306758,56045,56,6891,576641,0.011950,20220409,8.000000,18.0,18.0,0.0,...,666,55,10,677,752,455,179,52,88,90
306759,56045,56,6891,576641,0.011950,20220416,5.000000,18.0,18.0,0.0,...,666,55,10,677,752,455,179,52,88,90
306760,56045,56,6891,576641,0.011950,20220423,4.000000,18.0,18.0,0.0,...,666,55,10,677,752,455,179,52,88,90
306761,56045,56,6891,576641,0.011950,20220430,4.000000,18.0,18.0,0.0,...,666,55,10,677,752,455,179,52,88,90


### <font color=blue> Random forest.

In [17]:
def get_chunks(lst, n):
    """Return successive n-sized chunks from lst."""
    chunks = []
    for i in range(0, len(lst), n):
        chunks.append(lst[i:i + n])
    return chunks

chunks = get_chunks(list(df.Date.unique()), 4)

# idc = [2,10,18]    
idx = 18 #10

dates_test = chunks[idx]
dates_train = [x for x in df.Date.unique() if x not in dates_test]

df_train = df[df.Date.isin(dates_train)]
df_test = df[df.Date.isin(dates_test)]

feats = list(df_filt.columns) + ['Proj_inc_deaths','Cum_deaths_tm1',
                                'Naive_inc_deaths','Pop','Ratio','State_fips']

X_train = df_train[feats]
X_test = df_test[feats]

y_train = df_train['True_inc_deaths']
y_test = df_test['True_inc_deaths']
y_naive = df_test['Naive_inc_deaths']

In [18]:
from sklearn.ensemble import RandomForestRegressor
# train time for random split at 0.1 test_size? 2 min
clf = RandomForestRegressor(random_state=666, max_depth=6)
clf.fit(X_train, y_train)

RandomForestRegressor(max_depth=6, random_state=666)

In [19]:
from epsampling.utils import get_performance

y_pred = clf.predict(X_test)

model_names = ['Deaths_pred', 'Deaths_naive']
model_preds = [y_pred, y_naive]

metrics_dict = get_performance(model_names, model_preds, y_test, y_naive)
metrics_dict

{'Deaths_pred': {'MAE': 1.8181705695791586,
  'MSE': 21.818962582309588,
  'RMSE': 4.671077240028213,
  'r2': 0.6130040203432526,
  'relMAE': 2.0216441082637093},
 'Deaths_naive': {'MAE': 1.8284486014710162,
  'MSE': 22.9026792245397,
  'RMSE': 4.78567437510532,
  'r2': 0.59378248393665,
  'relMAE': 1.0}}

### <font color=blue> Do _naive_ metrics vary a lot across counties?

In [ ]:
from epsampling.utils import get_performance
import pprint

county_res_dict = {}

fipss = df.Fips.unique()

for fips in tqdm(df.Fips.unique(), total = len(fipss)):
    
    df_fips = df[df.Fips==fips]
    
    y_naive = df_fips['Naive_inc_deaths'].values
    y_test = df_fips['True_inc_deaths'].values

    model_names = ['Naive_inc_deaths'] #,'Deaths_true']
    model_preds = [y_naive]
    
    metrics_dict = get_performance(model_names, model_preds, y_test, y_naive)
    county_res_dict[fips] = list(metrics_dict.values())[0]
    
df_res = pd.DataFrame([[county,metric,value] 
                       for county,d in county_res_dict.items() 
                       for metric,value in d.items()], 
                       columns = ['Fips','metric','value'])

df_res = df_res.pivot(index='Fips',columns='metric',values='value')
df_res.drop('relMAE',axis=1,inplace=True)

df_res.reset_index(inplace=True)
df_res.columns.name = None
df_res.index.name = None
df_res

In [ ]:
print(f'\n\n* MAE * \nvariance: {round(df_res.MAE.var(),3)}\n',df_res.MAE.describe(),
      f'\n\n* MSE * \nvariance: {round(df_res.MSE.var(),3)}\n',df_res.MSE.describe(),
      f'\n\n* RMSE * \nvariance: {round(df_res.RMSE.var(),3)}\n',df_res.RMSE.describe(),
      f'\n\n* r2 * \nvariance: {round(df_res.r2.var(),3)}\n',df_res.r2.describe())